# Exploration of the scraping model for whisky reviews

The plan is to store reviews in Azure blob storage and see if we can make sense of them

Connect to database

In [2]:
%load_ext autoreload
%autoreload 2
import whiskymetrics.database as db
from sqlalchemy.sql import select
import pandas as pd

review = db.create_metadata().tables['review']
conn = db.get_connection()

s = select([review.c.id, review.c.link])
print(s)

url_df = pd.read_sql(sql=s,con=conn)

This is where we explore how to get reviews from reddit into a json/xml file. 

### Method 1: Prentend to be a browser

In [36]:
import requests
from time import sleep
headers={'User-agent': 'Mozilla/5.0'}

my_links = [m[0] for m in moo[0:9]]
print(my_links)
json_str = '[\n'
for link in my_links:
    resp = requests.get(link+'.json',headers,proxies={'http':'http://177.54.158.201:8080'})
    print(resp.status_code)
    if resp.status_code == '200':
        print(resp.text)
        json_str += resp.text
    sleep(2)
        
json_str+='\n]'

print(json_str)

200
200
429
200
200
429
429
429
429
[

]


The problem with this method is that we trigger reddit's 429. Maybe Praw will provide a better solution

### Method 2 Use PRAW

In [3]:

my_links = url_df.sample(n=50)
my_links.head()

NameError: name 'url_df' is not defined

In [122]:
import praw
from numpy import nan

reddit = praw.Reddit(client_secret='vPPOuaLhteQcDgMUv0v5Qm0p_lM',user_agent='reviewbot (dev) v0.0.1 (by /u/TheJollyReaver)',client_id='4kmOMwhAe_AjMA')

fullnames = []
for link in my_links['link']:
    try:
        fullnames.append(reddit.submission(url = link).comments[0].fullname)
    except:
        fullnames.append(nan)
        
            
    
my_links = my_links.assign(fullname=fullnames).dropna()
my_links.head()

,id,link,fullname
983,984,https://www.reddit.com/r/Scotch/comments/4aop9...,t1_d125f9k
1283,1284,http://www.reddit.com/r/Scotch/comments/2rsxiu...,t1_cniyb0j
3035,3036,http://www.reddit.com/r/Scotch/comments/19hi6v...,t1_c8o2bo4
1763,1764,http://www.reddit.com/r/Scotch/comments/2pee5z...,t1_cmvwmao
2217,2218,http://www.reddit.com/r/Scotch/comments/36b4k1...,t1_crcdi6l


In [128]:
my_links
text = []

for m in reddit.info(list(my_links['fullname'])):
    text.append(m.body)
    
my_links = my_links.assign(review_text=text).drop('link',1)
my_links.head()

,id,fullname,review_text
983,984,t1_d125f9k,**Bruichladdich Octomore 6.1.** 57% abv. Taste...
1283,1284,t1_cniyb0j,*****\n**ROUND TWO - Session Four - a more or ...
3035,3036,t1_c8o2bo4,"I understand this has been reviewed massively,..."
1763,1764,t1_cmvwmao,The 12 and up is where it's at with this brand...
2217,2218,t1_crcdi6l,Greetings Scotchit! You know those really entr...


In [1]:
from whiskymetrics.etl import load

tbl = db.create_metadata().tables['review_text']
load(tbl,my_links)

NameError: name 'db' is not defined

NameError: name 'tl' is not defined